<a href="https://colab.research.google.com/github/ssheggrud/Mod_20_Project/blob/05_Pooja/Mod_20_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
#!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz

!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InR

In [6]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("FinalProject").getOrCreate()

In [45]:
# Read in data from S3 Buckets
#from pyspark import SparkFiles
#url = "https://databootcamp-final-05.s3.amazonaws.com/Resources/2005_expenditure.csv"
#df2 = spark.read.options(inferSchema='True',delimiter=',', header=True).csv(url)
#df2.show()

from pyspark import SparkFiles
url = "https://databootcamp-final-05.s3.amazonaws.com/Resources/Individual+Contributions+2013+(UF).csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("Individual+Contributions+2013+(UF).csv"), sep=",", header=True )
df.show()

+--------+--------+-------+--------+------------------+---------+------+--------+------+----------+--------+---------+----------+-------------------+------+-----+-------+---------+---------+-----------------+-----+-----+--------------+--------------------+--------+------------------+---------------+--------+----+---------+--------+----------+--------+----------+--------+--------+--------+-------+-----+------+----------+----------+----------+----------+--------+---------+---------+--------+---------+------+-------+----------+
|ELECTION|OFFICECD|RECIPID|CANCLASS|         RECIPNAME|COMMITTEE|FILING|SCHEDULE|PAGENO|SEQUENCENO|   REFNO|     DATE|REFUNDDATE|               NAME|C_CODE|STRNO|STRNAME|APARTMENT|BOROUGHCD|             CITY|STATE|  ZIP|    OCCUPATION|             EMPNAME|EMPSTRNO|        EMPSTRNAME|        EMPCITY|EMPSTATE|AMNT|MATCHAMNT|PREVAMNT|PAY_METHOD|INTERMNO|INTERMNAME|INTSTRNO|INTSTRNM|INTAPTNO|INTCITY|INTST|INTZIP|INTEMPNAME|INTEMPSTNO|INTEMPSTNM|INTEMPCITY|INTEMPST|INTO

In [46]:
df.dtypes

[('ELECTION', 'string'),
 ('OFFICECD', 'string'),
 ('RECIPID', 'string'),
 ('CANCLASS', 'string'),
 ('RECIPNAME', 'string'),
 ('COMMITTEE', 'string'),
 ('FILING', 'string'),
 ('SCHEDULE', 'string'),
 ('PAGENO', 'string'),
 ('SEQUENCENO', 'string'),
 ('REFNO', 'string'),
 ('DATE', 'string'),
 ('REFUNDDATE', 'string'),
 ('NAME', 'string'),
 ('C_CODE', 'string'),
 ('STRNO', 'string'),
 ('STRNAME', 'string'),
 ('APARTMENT', 'string'),
 ('BOROUGHCD', 'string'),
 ('CITY', 'string'),
 ('STATE', 'string'),
 ('ZIP', 'string'),
 ('OCCUPATION', 'string'),
 ('EMPNAME', 'string'),
 ('EMPSTRNO', 'string'),
 ('EMPSTRNAME', 'string'),
 ('EMPCITY', 'string'),
 ('EMPSTATE', 'string'),
 ('AMNT', 'string'),
 ('MATCHAMNT', 'string'),
 ('PREVAMNT', 'string'),
 ('PAY_METHOD', 'string'),
 ('INTERMNO', 'string'),
 ('INTERMNAME', 'string'),
 ('INTSTRNO', 'string'),
 ('INTSTRNM', 'string'),
 ('INTAPTNO', 'string'),
 ('INTCITY', 'string'),
 ('INTST', 'string'),
 ('INTZIP', 'string'),
 ('INTEMPNAME', 'string'),
 (

In [47]:
# Remove multiple columns
list = ['COMMITTEE', 'FILING', 'SCHEDULE', 'PAGENO', 'SEQUENCENO','REFNO', 'STRNO', 'STRNAME', 'APARTMENT', 'OCCUPATION', 'EMPNAME', 'EMPSTRNO', 'EMPSTRNAME',
                'EMPCITY','EMPSTATE','INTERMNO', 'INTERMNAME', 'INTSTRNO', 'INTSTRNM', 'INTSTRNM', 'INTAPTNO',
                'INTCITY', 'INTST','INTZIP', 'INTEMPNAME', 'INTEMPSTNO', 'INTEMPSTNM', 'INTEMPCITY', 'INTEMPST', 'INTOCCUPA' ,'PURPOSECD', 'EXEMPTCD','ADJTYPECD', 'RR_IND', 'SEG_IND','INT_C_CODE'] 
df = df.drop(*list)
df.show()

+--------+--------+-------+--------+------------------+---------+----------+-------------------+------+---------+-----------------+-----+-----+----+---------+--------+----------+
|ELECTION|OFFICECD|RECIPID|CANCLASS|         RECIPNAME|     DATE|REFUNDDATE|               NAME|C_CODE|BOROUGHCD|             CITY|STATE|  ZIP|AMNT|MATCHAMNT|PREVAMNT|PAY_METHOD|
+--------+--------+-------+--------+------------------+---------+----------+-------------------+------+---------+-----------------+-----+-----+----+---------+--------+----------+
|    2013|       5|   1008|       P|Peterson, Daniel P|3/11/2012|      null|   Haywood, David B|   IND|        X|            Bronx|   NY|10458|  20|       20|       0|         4|
|    2013|       5|   1008|       P|Peterson, Daniel P| 3/6/2012|      null|      Herlihy, Neil|   IND|        Z|       Hunnington|   NY|11746|  25|        0|       0|         2|
|    2013|       5|   1008|       P|Peterson, Daniel P| 3/5/2012|      null|       Iocca, Karen|   IND|  

In [48]:
df1 = df \
.withColumnRenamed("ELECTION", "Election") \
.withColumnRenamed("RECIPID", "CanId") \
.withColumnRenamed("RECIPNAME", "CandidateName") \
.withColumnRenamed("DATE", "Date") \
.withColumnRenamed("NAME", "Name") \
.withColumnRenamed("DATE", "Date") \
.withColumnRenamed("C_CODE", "ContributerType") \
.withColumnRenamed("BOROUGHCD", "BoroughName") \
.withColumnRenamed("CITY", "City") \
.withColumnRenamed("STATE", "State") \
.withColumnRenamed("ZIP", "Zip") \
.withColumnRenamed("AMNT", "Amount") \
.withColumnRenamed("MATCHAMNT", "MatchAmt") \
.withColumnRenamed("PREVAMNT", "PrevAmt") \
.withColumnRenamed("PAY_METHOD", "PayMethod") 

df1.printSchema()


root
 |-- Election: string (nullable = true)
 |-- OFFICECD: string (nullable = true)
 |-- CanId: string (nullable = true)
 |-- CANCLASS: string (nullable = true)
 |-- CandidateName: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- REFUNDDATE: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- ContributerType: string (nullable = true)
 |-- BoroughName: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Zip: string (nullable = true)
 |-- Amount: string (nullable = true)
 |-- MatchAmt: string (nullable = true)
 |-- PrevAmt: string (nullable = true)
 |-- PayMethod: string (nullable = true)



In [53]:
# changing datatypes of columns
df2 = df1.withColumn("Election",df1["Election"].cast("Integer")) \
  .withColumn("OFFICECD", df1["OFFICECD"].cast("Integer")) \
  .withColumn("CanId", df1["CanId"].cast("Integer")) \
  .withColumn("Zip", df1["Zip"].cast("Integer")) \
  .withColumn("Amount", df1["Amount"].cast("Float")) \
  .withColumn("MatchAmt", df1["MatchAmt"].cast("Float")) \
  .withColumn("PrevAmt", df1["PrevAmt"].cast("Float")) \
  .withColumn("Date", df1["Date"].cast("Date")) \
  .withColumn("REFUNDDATE", df1["REFUNDDATE"].cast("Date")) 
  
df2.printSchema()

root
 |-- Election: integer (nullable = true)
 |-- OFFICECD: integer (nullable = true)
 |-- CanId: integer (nullable = true)
 |-- CANCLASS: string (nullable = true)
 |-- CandidateName: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- REFUNDDATE: date (nullable = true)
 |-- Name: string (nullable = true)
 |-- ContributerType: string (nullable = true)
 |-- BoroughName: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Zip: integer (nullable = true)
 |-- Amount: float (nullable = true)
 |-- MatchAmt: float (nullable = true)
 |-- PrevAmt: float (nullable = true)
 |-- PayMethod: string (nullable = true)



In [54]:
df3 = df2.filter((df.OFFICECD=="1") & (df2.CANCLASS=="P"))

In [55]:
df3.show()

+--------+--------+-----+--------+--------------------+----+----------+-------------------+---------------+-----------+--------------+-----+-----+------+--------+-------+---------+
|Election|OFFICECD|CanId|CANCLASS|       CandidateName|Date|REFUNDDATE|               Name|ContributerType|BoroughName|          City|State|  Zip|Amount|MatchAmt|PrevAmt|PayMethod|
+--------+--------+-----+--------+--------------------+----+----------+-------------------+---------------+-----------+--------------+-----+-----+------+--------+-------+---------+
|    2013|       1| 1286|       P|Coenen, Jr., Kevin P|null|      null|    Coenen, Kevin P|            CAN|          Z|     Nesconset|   NY|11767|  10.0|     0.0|    0.0|        4|
|    2013|       1| 1286|       P|Coenen, Jr., Kevin P|null|      null|     Walter, Dorsey|            IND|          M|      New York|   NY|10150| 175.0|   175.0|    0.0|        4|
|    2013|       1| 1286|       P|Coenen, Jr., Kevin P|null|      null|    Coenen, Kevin P|    